In [1]:
%pylab inline
from numpy.lib.recfunctions import append_fields


Populating the interactive namespace from numpy and matplotlib


In [2]:
### Excluding the AGB stars with dust spheres around.
x = np.load("../data/GDR3/gaiadr3_0ext.npy")
cut = (x['log_lum'] <5.) & (x['log_lum'] >3.) & (x['log_teff'] <3.7) & (x['gaia_g'] > 0.5)
print(len(x), len(x[cut]))

8102858 322023


In [3]:
x = np.load("../data/TMASS/2mass_0ext.npy")
print(x.dtype.names)

('meh_ini', 'log_age', 'm_ini', 'm_act', 'log_lum', 'log_teff', 'log_grav', 'tmass_j', 'tmass_h', 'tmass_ks')


In [4]:
avs = np.array([0,1,2,3,5,10,20])
x = []
for av in avs:
    temp = np.load("../data/TMASS/2mass_%dext.npy" %(av))
    # we exclude the AGB dust stars from the averaging
    temp = temp[~cut]
    temp = append_fields(temp,"parsec_index",np.zeros(shape = len(temp),dtype = np.int32),usemask=False)
    x.append(temp)
print(len(x[0]),x[0].dtype)

7780835 [('meh_ini', '<f8'), ('log_age', '<f8'), ('m_ini', '<f8'), ('m_act', '<f8'), ('log_lum', '<f8'), ('log_teff', '<f8'), ('log_grav', '<f8'), ('tmass_j', '<f8'), ('tmass_h', '<f8'), ('tmass_ks', '<f8'), ('parsec_index', '<i4')]


In [5]:
for name in x[0].dtype.names:
    print(name,x[0][name],len(np.unique(x[0][name])))

meh_ini [-1.49213 -1.49213 -1.49213 ...  0.33631  0.33631  0.33631] 75
log_age [ 6.6      6.6      6.6     ... 10.12004 10.12004 10.12004] 177
m_ini [0.09       0.0956148  0.1        ... 5.80000019 5.99999952 6.19999933] 6067091
m_act [0.09  0.096 0.1   ... 0.94  0.961 0.962] 53226
log_lum [-1.531 -1.502 -1.481 ... -4.601 -4.603 -4.604] 10836
log_teff [3.5276 3.5305 3.5326 ... 3.6298 3.6321 3.6319] 25411
log_grav [3.986 3.995 4.001 ... 8.484 8.505 8.505] 11069
tmass_j [ 6.791  6.727  6.679 ... 14.661 14.671 14.672] 26750
tmass_h [ 6.23   6.164  6.114 ... 14.077 14.089 14.09 ] 26609
tmass_ks [ 6.019  5.953  5.903 ... 13.931 13.945 13.946] 26531
parsec_index [0 0 0 ... 0 0 0] 1


In [6]:
def return_index_feh(feh):
        dfeh = 0.05
        offset = 1.5
        return(int((feh+offset)/dfeh))
min_value = min(x[0]['meh_ini'])
max_value = max(x[0]['meh_ini'])
min_index_feh = return_index_feh(min_value)
max_index_feh = return_index_feh(max_value)
stretch = max_index_feh - min_index_feh
print('teff values in parsec')
print('teff min max value: ', min_value, max_value )
print('teff min max index: ', min_index_feh, max_index_feh)
print('dimension cut into %d pieces' %(stretch))

def return_index_teff(teff):
        dteff = 0.02
        return(int(teff/dteff))
min_value = min(x[0]['log_teff'])
max_value = max(x[0]['log_teff'])
min_index_teff = return_index_teff(min_value)
max_index_teff = return_index_teff(max_value)
stretch = max_index_teff - min_index_teff
print('teff values in parsec')
print('teff min max value: ', min_value, max_value )
print('teff min max index: ', min_index_teff, max_index_teff)
print('dimension cut into %d pieces' %(stretch))

def return_index_lum(lum):
        dlum = 0.05
        offset = 5.
        return(int((lum+offset)/dlum))

min_value = min(x[0]['log_lum'])
max_value = max(x[0]['log_lum'])
min_index_lum = return_index_lum(min_value)
max_index_lum = return_index_lum(max_value)
stretch = max_index_lum - min_index_lum
print('lum values in parsec')
print('lum min max value: ', min_value, max_value )
print('lum min max index: ', min_index_lum, max_index_lum )
print('dimension cut into %d pieces' %(stretch))

def return_index(feh,teff,lum):
        """
        feh given in dex
        teff given in log teff
        lum given in log lum
        """
        index_feh = return_index_feh(feh)
        if index_feh < min_index_feh:
            index_feh = min_index_feh
        elif index_feh > max_index_feh:
            index_feh = max_index_feh

        index_teff = return_index_teff(teff)
        if index_teff < min_index_teff:
            index_teff = min_index_teff
        elif index_teff > max_index_teff:
            index_teff = max_index_teff
        index_teff *= 1000

        index_lum = return_index_lum(lum)
        if index_lum > max_index_lum:
            index_lum = max_index_lum
        elif index_lum < min_index_lum:
            index_lum = min_index_lum
        index_lum *= 1000 * 1000
        assert(index_feh >= 0)
        assert(index_teff >= 0)
        assert(index_lum >= 0)
        return (index_feh + index_teff + index_lum)

teff values in parsec
teff min max value:  -1.49213 0.33631
teff min max index:  0 36
dimension cut into 36 pieces
teff values in parsec
teff min max value:  2.4484 5.6843
teff min max index:  122 284
dimension cut into 162 pieces
lum values in parsec
lum min max value:  -4.604 6.24
lum min max index:  7 224
dimension cut into 217 pieces


In [7]:
print('indexing isochrones')
for i in range(len(x[0])):
    x[0]["parsec_index"][i] = return_index(x[0]["meh_ini"][i],x[0]["log_teff"][i],x[0]["log_lum"][i])

indexing isochrones


In [8]:
# All need to be sorted by parsec_index
sort_ind = np.argsort(x[0]['parsec_index'], kind = 'mergesort')
for i in range(len(avs)):
    x[i] = x[i][sort_ind]

In [9]:
j_ext = np.zeros(shape=(len(x[0]),len(x)))
h_ext = np.zeros(shape=(len(x[0]),len(x)))
ks_ext = np.zeros(shape=(len(x[0]),len(x)))
for i,item in enumerate(x):
    print(i,item[0])
    j_ext[:,i] = item["tmass_j"]
    h_ext[:,i] = item["tmass_h"]
    ks_ext[:,i] = item["tmass_ks"]

0 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 14.711, 14.169, 14.023, 7181017)
1 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 15., 14.352, 14.141, 0)
2 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 15.289, 14.534, 14.258, 0)
3 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 15.576, 14.716, 14.376, 0)
4 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 16.15, 15.08, 14.611, 0)
5 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 17.575, 15.986, 15.199, 0)
6 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 20.386, 17.786, 16.369, 0)


In [10]:
# show the extinction law and on which parameters it is dependent
j_ext = j_ext - j_ext[:,0,None]
h_ext = h_ext - h_ext[:,0,None]
ks_ext = ks_ext - ks_ext[:,0,None]


In [11]:
ext = np.zeros(shape = (len(np.unique(x[0]["parsec_index"])),3,6),dtype = np.float32)
ext_std = np.zeros_like(ext)
parsec = np.zeros(shape =  (len(np.unique(x[0]["parsec_index"]))),dtype = x[0].dtype)
parsec_std = np.zeros(shape =  (len(np.unique(x[0]["parsec_index"]))),dtype = x[0].dtype)

In [12]:
parsec["parsec_index"] = np.unique(x[0]["parsec_index"])
parsec_std["parsec_index"] = np.unique(x[0]["parsec_index"])

In [13]:
indexing = np.hstack((0,np.searchsorted(x[0]['parsec_index'],parsec['parsec_index'], side='right')))
print(indexing)
for i in range(len(indexing)-1):
    if i%10000==0:
        print(i, len(indexing))
    lower = indexing[i]
    upper = indexing[i+1]
    #parsec['parsec_index'][i]
    #x[0]['parsec_index'][lower:upper]
    temp = x[0][lower:upper]
    j_temp = j_ext[lower:upper]
    h_temp = h_ext[lower:upper]
    ks_temp = ks_ext[lower:upper]
    for jtem in parsec.dtype.names:
        if jtem == "parsec_index":
            assert(parsec["parsec_index"][i] == temp["parsec_index"][0])
        else:
            parsec[jtem][i] = np.median(temp[jtem])
            parsec_std[jtem][i] = np.std(temp[jtem])
    for t in np.arange(6):
        ext[i,0,t] = np.median(j_temp[:,t+1])
        ext[i,1,t] = np.median(h_temp[:,t+1])
        ext[i,2,t] = np.median(ks_temp[:,t+1])
        
        ext_std[i,0,t] = np.std(j_temp[:,t+1])
        ext_std[i,1,t] = np.std(h_temp[:,t+1])
        ext_std[i,2,t] = np.std(ks_temp[:,t+1])


[      0       1       4 ... 7780833 7780834 7780835]
0 243239
10000 243239
20000 243239
30000 243239
40000 243239
50000 243239
60000 243239
70000 243239
80000 243239
90000 243239
100000 243239
110000 243239
120000 243239
130000 243239
140000 243239
150000 243239
160000 243239
170000 243239
180000 243239
190000 243239
200000 243239
210000 243239
220000 243239
230000 243239
240000 243239


In [14]:
np.save('ext_2mass.npy',ext)
np.save('ext_std_2mass.npy', ext_std)
np.save('parsec_2mass.npy', parsec)
np.save('parsec_std_2mass.npy', parsec_std)

In [15]:
# check that the stds are small and also that Galaxia falls onto those parsec indexes.